# Tratamento dos dados a serem utilizados para o desafio - 
    ## Machine Learning

In [2]:
import os
import sys
import pandas as pd
notebook_dir = os.getcwd() 
project_root = os.path.dirname(notebook_dir) 
source_path = os.path.join(project_root, 'source')

if source_path not in sys.path:
    sys.path.append(source_path)

from Funções_tratar_dados.tratar_dados import *

df = pd.read_csv(r'C:/Users/Brunin/Documents/Machine_Learning_random_Forest/dados/dados_brutos/Titanic2-Dataset.csv')
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Tratando dados e gerando um Csv com os dados tratados

In [4]:
# Tratamento da coluna Embarked e Sex 
coluna_embarked = df['Embarked'].isnull().sum()
print (f'Número de valores nulos na coluna Embarked: {coluna_embarked}')
valor_frequente = df['Embarked'].mode()[0]
df['Embarked'].fillna(valor_frequente, inplace=True)
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
print ("Tratamento concluido com sucesso ")
display(df.head())
# Salvando o DataFrame tratado em um novo arquivo CSV
pasta_destino_relativa = '../dados/dados_processados'
nome_arquivo = 'titanic_tratado_final.csv'
os.makedirs(pasta_destino_relativa, exist_ok=True)
caminho_absoluto = os.path.join(pasta_destino_relativa, nome_arquivo)
df.to_csv(caminho_absoluto, index=False)
print(f'Exportado com sucesso para: {os.path.abspath(caminho_absoluto)}')

Número de valores nulos na coluna Embarked: 0
Tratamento concluido com sucesso 


C:\Users\Brunin\AppData\Local\Temp\ipykernel_5324\1599328183.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(valor_frequente, inplace=True)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",NaN,22.0,1,0,A/5 21171,7.2500,NaN,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",NaN,38.0,1,0,PC 17599,71.2833,C85,NaN
2,3,1,3,"Heikkinen, Miss. Laina",NaN,26.0,0,0,STON/O2. 3101282,7.9250,NaN,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",NaN,35.0,1,0,113803,53.1000,C123,NaN
4,5,0,3,"Allen, Mr. William Henry",NaN,35.0,0,0,373450,8.0500,NaN,NaN


Exportado com sucesso para: c:\Users\Brunin\Documents\Machine_Learning_random_Forest\dados\dados_processados\titanic_tratado_final.csv


### Implementação do método random forest ( Floresta aleatória )

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score  , classification_report

features = ['Pclass' , 'Age' , 'SibSp' , 'Parch' ,'Fare' , 'Sex' , 'Embarked']
X = df[features]
Y = df['Survived']
X_train , X_test , Y_train , Y_test = train_test_split (X ,Y , test_size  = 0.3 , random_state = 42 , stratify = Y)
print (f'Shape de x_TREINO {X_train.shape }')
print (f'Shape de x_TESTE {X_test.shape }')
modelo_random_forest = RandomForestClassifier (
    n_estimators = 100 , max_depth = 5 , random_state = 42
)

print ('Iniciando o aperfeiçoamento do modelo...')
modelo_random_forest.fit (X_train , Y_train)
print ('Aperfeiçoamento concluído!')
previsoses = modelo_random_forest.predict(X_test)
acuracia = accuracy_score(Y_test , previsoses)
print (f'----------------------Resultados-------------------')
print (f'Acurácia do modelo : {acuracia *100 :.2f}')
print ('Relatório de Classificação :')
print (classification_report(Y_test , previsoses))
print ('-----------------------------------------------------')

variaveis_importantes = pd.Series (modelo_random_forest.feature_importances_ , index = features).sort_values(ascending = False)
print ('Variáveis mais importantes para o modelo :')
print (variaveis_importantes)

Shape de x_TREINO (623, 7)
Shape de x_TESTE (268, 7)
Iniciando o aperfeiçoamento do modelo...
Aperfeiçoamento concluído!
----------------------Resultados-------------------
Acurácia do modelo : 72.39
Relatório de Classificação :
              precision    recall  f1-score   support

           0       0.74      0.85      0.79       165
           1       0.69      0.51      0.59       103

    accuracy                           0.72       268
   macro avg       0.71      0.68      0.69       268
weighted avg       0.72      0.72      0.71       268

-----------------------------------------------------
Variáveis mais importantes para o modelo :
Fare        0.353913
Age         0.253609
Pclass      0.215629
SibSp       0.094172
Parch       0.082677
Sex         0.000000
Embarked    0.000000
dtype: float64
